In [1]:
# utilisation d'un dictionnaire pour représenter un fichier JSON d'intentions
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?", "سلام"],
              "responses": ["Salut à toi!", "Hello", "Comment vas tu?", "Salutations!", "Enchanté","مرحبا"],
             },
             {"tag": "age",
              "patterns": ["Quel âge as-tu?", "C'est quand ton anniversaire?", "Quand es-tu né?","كم عمرك الان؟"],
              "responses": ["J'ai 25 ans", "Je suis né en 1996", "Ma date d'anniversaire est le 3 juillet et je suis né en 1996", "03/07/1996","  03سبتمبر"]
             },
             {"tag": "date",
              "patterns": ["Que fais-tu ce week-end?",
"Tu veux qu'on fasse un truc ensemble?", "Quels sont tes plans pour cette semaine", "ماذا تفعل اخرالاسبوع؟"],
              "responses": ["Je suis libre toute la semaine", "Je n'ai rien de prévu", "Je ne suis pas occupé", "انا  غير مشغول"]
             },
             {"tag": "name",
              "patterns": ["Quel est ton prénom?", "Comment tu t'appelles?", "Qui es-tu?", " ما اسمك؟"],
              "responses": ["Mon prénom est Miki", "Je suis Miki", "Miki", "نسرين"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "Salut", "see ya", "adios", "cya", "الى اللقاء"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!", "الى اللقاء"],
             }
]}

In [2]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [4]:
# initialisation de lemmatizer pour obtenir la racine des mots
lemmatizer = WordNetLemmatizer()

# création des listes
words = []
classes = []
doc_X = []
doc_y = []

# parcourir avec une boucle For toutes les intentions
# tokéniser chaque pattern et ajouter les tokens à la liste words, les patterns et
# le tag associé à l'intention sont ajoutés aux listes correspondantes
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # ajouter le tag aux classes s'il n'est pas déjà là 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# lemmatiser tous les mots du vocabulaire et les convertir en minuscule
# si les mots n'apparaissent pas dans la ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

# trier le vocabulaire et les classes par ordre alphabétique et prendre le
# set pour s'assurer qu'il n'y a pas de doublons
words = sorted(set(words))
classes = sorted(set(classes))

In [5]:
print(words)

['adios', 'anniversaire', 'as-tu', 'bye', "c'est", 'ce', 'cette', 'comment', 'cya', 'ensemble', 'es-tu', 'est', 'fais-tu', 'fasse', 'forme', 'hello', 'la', 'né', 'plan', 'pour', 'prénom', "qu'on", 'quand', 'que', 'quel', 'quels', 'qui', 'roule', 'salut', 'see', 'semaine', 'sont', "t'appelles", 'te', 'ton', 'truc', 'tu', 'un', 'veux', 'week-end', 'ya', 'yo', 'âge', 'ça', 'اخرالاسبوع؟', 'اسمك؟', 'الان؟', 'اللقاء', 'الى', 'تفعل', 'سلام', 'عمرك', 'كم', 'ما', 'ماذا']


In [6]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [7]:
print(doc_X)

['Hello', 'La forme?', 'yo', 'Salut', 'ça roule?', 'سلام', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'كم عمرك الان؟', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'ماذا تفعل اخرالاسبوع؟', 'Quel est ton prénom?', "Comment tu t'appelles?", 'Qui es-tu?', ' ما اسمك؟', 'bye', 'Salut', 'see ya', 'adios', 'cya', 'الى اللقاء']


In [8]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'age', 'date', 'date', 'date', 'date', 'name', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [9]:
# liste pour les données d'entraînement
training = []
out_empty = [0] * len(classes)

# création du modèle d'ensemble de mots
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    # marque l'index de la classe à laquelle le pattern atguel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    # ajoute le one hot encoded BoW et les classes associées à la liste training
    training.append([bow, output_row])

# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)

# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [10]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 500

In [11]:
# modèle Deep Learning
model = Sequential()
model.add(Dense(512, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dropout(0.4))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dropout(0.2))
model.add(Dropout(0.2))

model.add(Dense(output_shape, activation = "softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-5)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [12]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               28672     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dropout_4 (Dropout)         (None, 128)               0

In [13]:
# entraînement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 646ms/step - loss: 1.5491 - accuracy: 0.2500
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 1.5717 - accuracy: 0.3750
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 1.6340 - accuracy: 0.2083
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 1.5977 - accuracy: 0.1667
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 1.6140 - accuracy: 0.2083
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 1.4909 - accuracy: 0.5000
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 1.5366 - accuracy: 0.3333
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 1.5134 - accuracy: 0.4583
Epoch 9/200
1/1 [==============================] - 0s 8ms/step - loss: 1.4881 - accuracy: 0.5000
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 1.4911 - accuracy: 0.3750
Epoch 11/200
1/1 [===

In [14]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# lancement du chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

سلام
Salutations!
سلام
Salutations!
سلام
مرحبا
  الى اللقاء
à plus tard
              الى اللقاء
C'était sympa de te parler
              الى اللقاء
à plus tard
